В этой тетрадке сначала тестируем accuracy и f1 модели отдельно по классам (meaningfulness, grammar, emotionality), затем считаем эти параметры для сгенерированных поэм. 

# Тестируем accuracy и f1 по классам

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_name_or_path = 'numblilbug/rubert-cased-poem-evalutation' 

model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
true_test_list = [[int(elem) for elem in sublist] for sublist in dataset['test']['labels']]

In [ ]:
test_classes = []
for i in dataset['test']['text']:
    inputs = tokenizer(i, return_tensors="pt",padding=True, truncation=True,max_length=50, add_special_tokens = True)
    outputs = model(**inputs)
    predicted_logits = outputs.logits
    predicted_classes = (predicted_logits > 0.6).int()
    test_classes.append(predicted_classes.tolist()[0])

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def calculate_scores_per_class(true_list, pred_list):
    num_classes = len(true_list[0])
    accuracy_per_class = []
    f1_scores_per_class = []

    for i in range(num_classes):
        true_class = [item[i] for item in true_list]
        pred_class = [item[i] for item in pred_list]

        accuracy = accuracy_score(true_class, pred_class)
        f1_score_class = f1_score(true_class, pred_class)

        accuracy_per_class.append(accuracy)
        f1_scores_per_class.append(f1_score_class)

    return accuracy_per_class, f1_scores_per_class



accuracy_per_class, f1_scores_per_class = calculate_scores_per_class(true_test_list, test_classes)
print("Accuracy per class:", accuracy_per_class)
print("F1 Scores per class:", f1_scores_per_class)

# Определяем критерии для сгенерированных поэм для дальнейшего сравнения моделей

In [ ]:
df_gen = pd.read_csv('/kaggle/input/gen-poems/poems_gen - 1.csv')
df_gen = df_gen[0:100]
df_gen = df_gen.sort_values('model')
df_gen = df_gen.reset_index()
df_gen = df_gen.drop('index', axis = 1)

In [ ]:
gen_text = df_gen['text'].to_list()

In [ ]:
gen_classes = []
for i in gen_text:
    inputs = tokenizer(i, return_tensors="pt",padding=True, truncation=True,max_length=50, add_special_tokens = True)
    outputs = model(**inputs)
    predicted_logits = outputs.logits
    predicted_classes = (predicted_logits > 0.6).int()
    gen_classes.append(predicted_classes.tolist()[0])

In [ ]:
transposed_list = list(map(list, zip(*gen_classes)))

df_gen['meaningfulness'], df_gen['grammar'], df_gen['emotionality'] = transposed_list


In [ ]:
df_gen.to_csv('gen_poems.csv')